Imports, secrets, and setup

In [3]:
import csv
import requests
from dotenv import load_dotenv
import os
from datetime import date

load_dotenv()
NOTION_API = os.getenv('NOTION_API')
NOTION_DATABASE_ID = os.getenv('NOTION_DATABASE_ID')

class Entry:
    def __init__(self, name, height, imageUrl, categories, sources):
        self.name: str = name
        self.height: int = height
        self.imageUrl: str = imageUrl
        self.categories: str = categories
        self.sources: str = sources

    def toList(self) -> list[str | int]:
        return [self.name, self.height, self.imageUrl, self.categories, self.sources]

Collect entries from Notion DB

In [4]:
data: list[Entry] = []

headers = {
    'Authorization': f"Bearer {NOTION_API}",
    'Content-Type': 'application/json',
    'Notion-Version': '2022-06-28',
}

not_done = True
start_cursor = ''
while (not_done):
    body = {}
    if start_cursor != '':
        body = {
            'start_cursor': start_cursor
        }

    response = requests.post(f"https://api.notion.com/v1/databases/{NOTION_DATABASE_ID}/query", headers=headers, json=body)
    if response.status_code != 404:
        json = response.json()

        for row in json['results']:
            props = row['properties']

            name = props['Name']['title'][0]['plain_text']
            height = props['Height (cm)']['number']
            imageUrl = props['Image URL']['url']
            categories = f"[{','.join(list(map(lambda x: x['name'], props['Tags']['multi_select'])))}]"
            sources = f"[{','.join(list(map(lambda x: x['name'], props['Source']['multi_select'])))}]"

            data.append(Entry(name, height, imageUrl, categories, sources))

        if json['has_more']:
            start_cursor = json['next_cursor']
        else:
            not_done = False

print(f"fetched {len(data)} entries!")

fetched 190 entries!


Save entries to csv

In [6]:
# only save entries if they have a height and img
save_if_have_height_and_img: bool = True


# filter entries if needed (only get if they have a height)
if save_if_have_height_and_img:
    new_data = []
    for entry in data:
        if entry.height != None and entry.imageUrl != None:
            new_data.append(entry)

    data = new_data

# save to a dated csv
with open(f"data/notion-data-{date.today()}.csv", 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    writer.writerow(['Name', 'Height', 'Image', 'Category', 'Source'])   
    writer.writerows([entry.toList() for entry in data])

print(f"saved {len(data)} entries!")

saved 23 entries!
